# Технический скрипт для домашки 1

Скачиваем данные о ценных бумагах. 

```
pip install yfinance
```

In [83]:
import pandas as pd
import re

# компании из индекса 
df = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers_sp = list(df.Symbol.values)

In [84]:
df2 = pd.read_html('https://en.wikipedia.org/wiki/NASDAQ-100')[2]
tickers_nasdaq = list(df2.Ticker.values)

In [85]:
df3 = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average')[1]
tickers_dji = list(df3.Symbol.str.split(':').apply(lambda w: re.sub('\xa0', '', w[-1])).values)

In [86]:
tickers = list(set(tickers_sp + tickers_nasdaq + tickers_dji))
len(tickers)

525

In [62]:
import yfinance as yf
import datetime

start = datetime.datetime(2010,1,1)
end = datetime.datetime(2021,1,1)
data = yf.download(tickers, start=start, end=end)

[*********************100%***********************]  525 of 525 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


In [63]:
data = data['Adj Close']
data.tail()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZM,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-27,97.269997,11.39,146.830002,378.557434,97.160004,103.574806,298.609985,99.989998,223.320007,437.100006,...,43.217411,43.450001,15.82,75.260002,93.370003,134.240005,273.339996,32.098629,252.330002,146.919998
2020-07-28,95.300003,11.77,146.190002,372.338684,96.709999,101.692535,294.890015,99.860001,222.179993,432.260010,...,42.707470,43.500000,15.21,74.339996,92.750000,134.160004,271.179993,31.950068,252.179993,145.979996
2020-07-29,97.099998,11.40,148.210007,379.475800,97.010002,102.887627,299.100006,102.519997,224.380005,436.299988,...,43.178185,44.790001,15.86,77.169998,94.830002,139.580002,273.850006,33.049404,252.389999,147.589996
2020-07-30,95.930000,11.18,152.869995,384.067535,96.040001,101.752289,298.910004,101.529999,223.979996,438.880005,...,41.059971,45.240002,16.32,74.669998,91.639999,138.300003,274.950012,32.465073,250.000000,149.110001
2020-07-31,96.330002,11.12,150.139999,424.275024,94.910004,99.780388,299.940002,100.639999,224.779999,444.320007,...,41.265911,44.599998,16.65,72.980003,91.050003,134.860001,280.750000,32.158054,253.910004,151.679993


In [68]:
# избавимся от пустых тикеров
isnull = data.isnull().sum()/data.shape[0]
bad_tickers = list(isnull[isnull == 1].index)
data.drop(bad_tickers, axis=1, inplace=True)

In [71]:
data.isnull().sum().sort_values()[-50:]/data.shape[0]

FBHS    0.115293
XYL     0.122821
APTV    0.132726
PSX     0.171949
SPLK    0.173930
AMCR    0.181062
FB      0.182250
TT      0.184628
NOW     0.193740
WDAY    0.222662
FANG    0.222662
ABBV    0.243661
NCLH    0.248415
ZTS     0.251981
IQV     0.278526
COTY    0.288035
NWS     0.289620
NWSA    0.289620
CDW     0.291997
TWTR    0.328843
ALLE    0.331616
HLT     0.338352
PAYC    0.371632
JD      0.381933
ANET    0.385895
INFO    0.389461
SYF     0.400951
CFG     0.416006
KEYS    0.423138
QRVO    0.443344
WRK     0.490491
KHC     0.493265
PYPL    0.493265
UA      0.494453
HPE     0.522583
FTV     0.593106
LW      0.629160
IR      0.678685
DOCU    0.774168
MRNA    0.835578
NLOK    0.859746
FOXA    0.860539
FOX     0.860935
DOW     0.862916
ZM      0.871236
CTVA    0.881141
HWM     0.927892
VIAC    0.939382
CARR    0.962758
OTIS    0.962758
dtype: float64

In [72]:
data.to_csv('price.tsv', sep='\t')

In [90]:
df['S&P-500'] = True
df.set_index('Symbol', inplace=True)

df2['NASDAQ'] = True
df2.set_index('Ticker', inplace=True)

df = df.join(df2, how='outer')

df3['DJI'] = True
df3['Symbol'] = df3['Symbol'].apply(lambda w: re.sub('\xa0', '', w.split(':')[-1]))
df4 = df3[['Symbol', 'DJI', 'Company']].set_index('Symbol')

df = df.join(df4, how='outer', rsuffix='dj')

In [91]:
df.to_csv('information.tsv', sep='\t')

In [92]:
df.shape

(525, 13)

In [95]:
df3.columns

Index(['Company', 'Exchange', 'Symbol', 'Industry', 'Date added', 'Notes',
       'Index weighting(April 30, 2020)', 'DJI'],
      dtype='object')

In [99]:
df3[['Symbol', 'Index weighting(April 30, 2020)']].set_index('Symbol').to_dict()['Index weighting(April 30, 2020)']

{'MMM': '4.35%',
 'AXP': '2.68%',
 'AAPL': '8.01%',
 'BA': '3.87%',
 'CAT': '3.34%',
 'CVX': '2.63%',
 'CSCO': '1.21%',
 'KO': '1.31%',
 'DOW': '1.04%',
 'XOM': '1.32%',
 'GS': '5.29%',
 'HD': '6.17%',
 'IBM': '3.58%',
 'INTC': '1.72%',
 'JNJ': '4.18%',
 'JPM': '2.72%',
 'MCD': '5.23%',
 'MRK': '2.25%',
 'MSFT': '4.94%',
 'NKE': '2.45%',
 'PFE': '1.06%',
 'PG': '3.26%',
 'RTX': '1.89%',
 'TRV': '2.95%',
 'UNH': '8.01%',
 'VZ': '1.62%',
 'V': '5.06%',
 'WMT': '3.44%',
 'WBA': '1.26%',
 'DIS': '3.12%'}